Since joins are very common when working with databases, SQL provides its own JOIN operator for this purpose. We can use this operator in a variety of ways to achieve a variety of join types. To emulate the equi-joins we saw on the previous slides, we can use the JOIN operator with two different keywords:

- JOIN ... USING:<br>
Specifying a field of attribute to test for equality
- JOIN ... ON:<br>
Specifying a condition

## Handling cursor data

As you've seen in the last problem, the data from SQL queries in Psycopg2 is returned in form of Python lists. In the last problem, you requested the full Star and Planet table, which returned a list of n tuples of length m, where m is the number of columns and n is the number of rows in these tables.

A list of tuples cannot be used in the same way as e.g. a 2D Numpy array. For example, the following method of indexing to access the first element will not work:


``` Python
a = [(1, 2, 3), (4, 5, 6)]
print(a[0, 0])
```

Instead, we have to use the [] operator twice: first to access the first list element, i.e. the first tuple, and then to access the first element in that tuple:

``` Python
a = [(1, 2, 3), (4, 5, 6)]
print(a[0][0])
```


Using this indexing method, we can then access every individual data element. This allows us to, e.g. extract entire columns of the data by looping over the rows. The following code snippet shows an example which extracts the t_eff column from the full Star table and appends it to a new list:

``` Python
import psycopg2
conn = psycopg2.connect(dbname='db', user='grok')
cursor = conn.cursor()
cursor.execute('SELECT * FROM Star')
records = cursor.fetchall()

t_eff = []
for row in records:
  t_eff.append(row[1])
print(t_eff)
```



## Data types in SQL and Python

Now we've seen how to work with query results, we can have a closer look at the data itself. In the previous activity, we learned about different data types in SQL when we were setting up tables.

How do these SQL data types get converted into Python types?

Let's have a look at the Planet table's data types. We can use a query which selects all columns but only a single row:
``` sql
SELECT * FROM Planet LIMIT 1;
```

In Python, this query will return a list containing a single tuple. We can loop over the entries of this tuple and call the type function to determine the data types:
``` Python
import psycopg2
conn = psycopg2.connect(dbname='db', user='grok')
cursor = conn.cursor()
cursor.execute('SELECT * FROM Planet LIMIT 1;')
records = cursor.fetchall()

for col in records[0]:
    print(type(col))
```

The type conversion of these types is straight-forward: SQL's SMALLINT and INTEGER get converted to Python integers, CHAR and VARCHAR to Python strings, and FLOAT to Python floats.

Check out the Psycopg2 documentation when you want to learn about type conversion in more detail. 


## Writing data into NumPy arrays

Once we have the numerical data from the database in Python, we can write them into NumPy arrays.
Since we're often dealing with data of different types in databases, it is important to remember that while Python lists and tuples can hold data of different types, NumPy arrays cannot.

To convert a Python list into a simple NumPy array, we must ensure that the list only contains data of one type. Other than that, SQL results can easily be loaded into NumPy arrays:

``` Python
import psycopg2
import numpy as np
conn = psycopg2.connect(dbname='db', user='grok')
cursor = conn.cursor()
cursor.execute('SELECT radius FROM Star;')
records = cursor.fetchall()

array = np.array(records)
print(array.shape)
print(array.mean())
print(array.std())

```
(66, 1)

0.886863636364

0.237456527847

Once the data is stored in NumPy arrays, we have access to all of NumPy's functionality to manipulate and analyse our data. One thing that we can now easily do is for example calculating a median.

## A proper median

Write a function called column_stats which calculates the mean and median of a selected column in either Star or Planet table. For this, let your function take two string arguments:

    1 - the name of the table;
    2 - the name of the column.

and have it return the mean and median (in this order) of the selected column.
When you call your function on, for example, the t_eff column of the Star table, the function call and return should look like this:

``` Python
column_stats('Star', 't_eff')
(5490.681818181818, 5634.0)
```

You can compare your calculation with the pure SQL query:

```sql
SELECT AVG(t_eff) FROM Star;
```
```
+-----------------------+
|          avg          |
+-----------------------+
| 5490.6818181818181818 |
+-----------------------+
```

``` Python
import psycopg2
import numpy as np
conn = psycopg2.connect(dbname='db', user='grok')
cursor = conn.cursor()
def column_stats(indb, cols):
  cursor.execute('SELECT '+cols+' FROM '+indb+';')
  records = cursor.fetchall()
  array = np.array(records)
  return (array.mean(), np.median(array))
```


## SQL vs. Python

In this course you've learned two different approaches to dealing with data. Which you choose for a particular project depends on a variety of factors including the questions you're posing of the data or whether you're using a public database or catalogue.

We have seen that SQL is convenient to use for a lot of things – but exactly how convenient is it? Can we do the same thing in Python?

Let's go through a few problems in which we implement typical SQL queries from the previous activities in Python. We will start of with a simple query and add a new element in each problem.


## Simple queries in Python 1/3

Your first task is to replicate the following SQL query:

```sql
SELECT kepler_id, radius
FROM Star
WHERE radius > 1.0;
```

The data is stored in stars.csv, with the kepler_id in the first column and the radius in the last.

Write a function called query which takes the CSV filename as an argument and returns the data in a 2-column NumPy array. For example, this small CSV file:
```
stars.csv
10666592,6350,1.991
10682541,5339,0.847
10797460,5850,1.04
```
your query function should work as follows:

``` Python
query('stars.csv')
--> array([[  1.06665920e+07   1.99100000e+00]
       [  1.07974600e+07   1.04000000e+00]])
```
The numerical data gets automatically converted to floats in this procedure, don't worry if it doesn't look like the SQL output. 


``` Python
import numpy as np
def query(incsv):
  data = np.loadtxt(incsv, usecols=(0,2), delimiter=',',unpack=True)
  nn = np.where(data[1,:] > 1.0)
  hh = np.transpose(data[:,nn[0]])
  return hh

# You can use this to test your code
# Everything inside this if-statement will be ignored by the automarker
if __name__ == '__main__':
  # Compare your function output to the SQL query
  result = query('stars.csv')
```


## Simple queries in Python 2/3

Let's add another element to our query. Sort the resulting table in ascending order to match the result you would get with:

```sql
SELECT kepler_id, radius
FROM Star
WHERE radius > 1.0
ORDER BY radius ASC;
```

You can use your results from the last problem and then build up on that. Again, the function should be named query and it should take the filename as argument.

- Hint

You can use NumPy's argsort function to solve this problem. Take a look at how it works:

``` Python
import numpy as np
a = np.array([3, 1, 2, 0])
b = np.argsort(a)
print(b)
print(a[b])
```

It returns the indices of the unsorted array a in their new, sorted positions. You can pass this returned array b to the original array a to rearrange its entries. 

**Result**

``` Python
import numpy as np
def query(incsv):
  data = np.loadtxt(incsv, usecols=(0,2), delimiter=',',unpack=True)
  nn = np.where(data[1,:] > 1.0)
  hh = np.transpose(data[:,nn[0]])
  b = np.argsort(hh[:,1])
  return hh[b,:]
```

## Simple queries in Python 3/3

Let's add yet another element to our query. Join the Star table with the Planet table and calculate the size ratio, i.e. planet radius / star radius for each star-planet pair. Your query function should produce the same result as the SQL query:

```sql
SELECT p.radius/s.radius AS radius_ratio
FROM Planet AS p
INNER JOIN star AS s USING (kepler_id)
WHERE s.radius > 1.0
ORDER BY p.radius/s.radius ASC;
```

You can use your results from the last problem and then build up on that. The function must be named query, but this time it should take two filenames as arguments, for the stars and planets.

In planets.csv, the first column is the kepler_id and the second last column is the radius.

Your function should be a column vector of ratios, like this:

```Python
>>> query('stars.csv', 'planets.csv')
array([[  0.48798799],
       [  0.8260447 ],
       [  0.96209913],
       [  1.1556384 ],
       [  1.30403969],
       ...
```

**Hint**

You may need to use a nested loop to compare each Planet's kepler_id against each Star's kepler_id. Once you've found a match and the star's radius is larger than one, you can append the ratio to the results list or array.

```Python
# Write your query function here
import numpy as np
def query(incsv1, incsv2):
  data1 = np.loadtxt(incsv1, usecols=(0,2), delimiter=',',unpack=True)
  data2 = np.loadtxt(incsv2, usecols=(0,5), delimiter=',',unpack=True) 
  ratio = np.asarray([])
  for i in range(len(data1[1,:])):
    for j in range(len(data2[1,:])):
      if ((data1[0,i] == data2[0,j]) & (data1[1,i] > 1.0)) :
        ratio = np.append(ratio, [data2[1,j]/data1[1,i]])
  b = np.argsort(ratio)
  return ratio[b].reshape(len(ratio),1)

# You can use this to test your code
# Everything inside this if-statement will be ignored by the automarker
if __name__ == '__main__':
  # Compare your function output to the SQL query
  result = query('stars.csv', 'planets.csv')
  

```


## The right tool for every job

The last three problems showed that Python is straight-forward to use for simple queries, but gets a lot more difficult as queries become more complex. The last question on joins was especially hard to implement in Python, whereas it's relatively simple in SQL.

This shouldn't be surprising, as that's exactly what SQL is designed for and what we should use for these problems. There are good reasons though for why you might not want to drop Python completely for database-related work.

One important thing to consider is that SQL is developed for accessing data and the built-in functions support only basic mathematical operations. Beyond that it gets very complicated.

A good example for this is the calculation of the median, which we have done a couple of times in Python. There are no built-in functions for the median in SQL however, and doing it by hand in SQL gets pretty complicated. We haven't even covered enough SQL to understand how to implement a median, but if you're interested, check out this Postgresql article which shows examples of how a median could be implemented.


# Machine learning

## Introduction

In this activity, we're going to use decision trees to determine the redshifts of galaxies from their photometric colours. We'll use galaxies where accurate spectroscopic redshifts have been calculated as our gold standard. We will learn how to assess the accuracy of the decision trees predictions and have a look at validation of our model.

We will also have a quick look at how this problem might be approached without using machine learning. This will highlight some of the limitations of the classical approach and demonstrate why a machine learning approach is ideal for this type of problem.

If you want to run your code offline, you can download the full NumPy dataset for this activity here.

This activity is based on the scikit-learn example on Photometric Redshifts of Galaxies.



## Magnitudes and colours

We will be using flux magnitudes from the Sloan Digital Sky Survey (SDSS) catalogue to create colour indices. Flux magnitudes are the total flux (or light) received in five frequency bands (u, g, r, i and z).
image of sdss filters


<img src="files/week5/plot_sdss_filters_11.png",width=450, height=400 >


*The astronomical colour (or colour index)* is the difference between the magnitudes of two filters, i.e. *u - g* or *i - z*.

This index is one way to characterise the colours of galaxies. For example, if the *u-g* index is high then the object is brighter in ultra violet frequencies than it is in visible green frequencies.

Colour indices act as an approximation for the spectrum of the object and are useful for classifying stars into different types.

## What data do we need?

To calculate the redshift of a distant galaxy, the most accurate method is to observe the optical emission lines and measure the shift in wavelength. However, this process can be time consuming and is thus infeasible for large samples.

For many galaxies we simply don't have spectroscopic observations.

Instead, we can calculate the redshift by measuring the flux using a number of different filters and comparing this to models of what we expect galaxies to look like at different redshifts.

In this activity, we will use machine learning to obtain photometric redshifts for a large sample of galaxies. We will use the colour indices (u-g, g-i, r-i and i-z) as our input and a subset of sources with spectroscopic redshifts as the training dataset.


## Decision tree algorithms

Decision trees are a tool that can be used for both classification and regression. In this module we will look at regression, however, in the next module we will see how they can be used as classifiers.

Decision trees map a set of input features to their corresponding output targets. This is done through a series of individual decisions where each decision represents a node (or branching) of the tree.

The following figure shows the decision tree our proverbial robot tennis player Robi used in the lectures to try and decide whether to play tennis on a particular day.
Tennis decision tree

<img src="week5/tennis.png",width=450, height=400 >

Should we play tennis?

Each node represents a decision that the robot needs to make (or assess) to reach a final decision. In this example, the decision tree will be passed a set of input features (Outlook, Humidity and Wind) and will return an output of whether to play or not.


## Decision trees for regression

In decision trees for real-world tasks, each decision is typically more complex, involving measured values, not just categories.

Instead of the input values for humidity being Normal or High and wind being Strong or Weak we might see a percentage between 0 and 100 for humidity and a wind speed in km/hr for wind. Our decisions might then be humidity < 40% or wind < 5 km/hr.

The output of regression is a real number. So, instead of the two outputs of <font color='blue'>Play</font> and <font color='blue'>Don't Play </font>we have a probability of whether we will play that day.

The decision at each branch is determined from the training data by the decision tree learning algorithm. Each algorithm employs a different metric (e.g. `Gini impurity` or `information gain`) to find the decision that splits the data most effectively.

For now, just need to know that a decision tree is a series of decisions, each made on a single feature of the data. The end point of all the branches is a set of desired target values.


## Decision trees in Python

The inputs to our decision tree are the colour indices from photometric imaging and our output is a photometric redshift. Our training data uses accurate spectroscopic measurements.

The decision tree will look something like the following.

<img src="week5/decisiontree.png",width=450, height=400 >

We can see how our calculated colour indices are input as features at the top and through a series of decision nodes a target redshift value is reached and output.

We will be using the Python machine learning library `scikit-learn` which provides several machine learning algorithms.

The [`scikit-learn decision tree regression`](http://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html)  takes a set of input features and the corresponding target values, and constructs a decision tree model that can be applied to new data.

## Sloan Digital Sky Survey data

We have provided the Sloan data in NumPy binary format (.npy) in a file called sdss_galaxy_colors.npy. The Sloan data is stored in a NumPy structured array and looks like this:

|u 	|g 	|r 	|i 	|z 	|... 	|redshift
|---|:---:|:---:|:---:|:---:|:---:|:---:|
|19.84 	|19.53 |	19.47 	|19.18 	|19.11 	|... 	|0.54|
|19.86 	|18.66 |	17.84 	|17.39 	|17.14 	|... 	|0.16|
|... 	|... 	|... 	|... 	|... 	|... 	|...|
|18.00 	|17.81 	|17.77 	|17.73 	|17.73 	|... 	|0.47|

It also include spec_class and redshift_err columns we don't need in this activity. The data can be loaded using:

```python
import numpy as np
data = np.load('sdss_galaxy_colors.npy')
print(data[0])
```
The data[0] corresponds to the first row of the table above. Individual named columns can be accessed like this:

```python
import numpy as np
data = np.load('sdss_galaxy_colors.npy')
print(data['u'])
```

Each flux magnitude column can be accessed in the data array as data['u'], data['g'] etc. The redshifts can accessed with data['redshift']. 



## Features and Targets

Write a `get_features_targets` function that splits the training data into input `features` and their corresponding `targets`. In our case, the inputs are the 4 colour indices and our targets are the corresponding redshifts.

Your function should return a tuple of:

   - **features**: a NumPy array of dimensions m ⨉ 4, where m is the number of galaxies;
   - **targets**: a 1D NumPy array of length m, containing the redshift for each galaxy.

The data argument will be the structured array described on the previous slide. The u flux magnitudes and redshifts can be accessed as a column with data['u'] and data['redshift'].

The four features are the colours u - g, g - r, r - i and i - z. To calculate the first column of features, subtract the u and g columns, like this:

```python
import numpy as np
data = np.load('sdss_galaxy_colors.npy')
print(data['u'] - data['g'])
```

The features for the first 2 galaxies in the example data should be:
```python
[[ 0.31476  0.0571   0.28991  0.07192]
 [ 1.2002   0.82026  0.45294  0.24665]]
```

And the first 2 targets should be:
```python
[ 0.539301   0.1645703]
```

## Hint:
set up your features array with zeros

You can set up the features array with zeros and then set each column to the corresponding calculated feature.
```python
features = np.zeros((data.shape[0], 4))
features[:, 0] = data['u'] - data['g']
```
```python
import numpy as np
def get_features_targets(data):
  # complete this function
  features = np.transpose([data['u']-data['g'],data['g']-data['r'],data['r']-data['i'],data['i']-data['z']])
  target = data['redshift']
  return features,target

if __name__ == "__main__":
  # load the data
  data = np.load('sdss_galaxy_colors.npy')
  # call our function 
  features, targets = get_features_targets(data)
  # print the shape of the returned arrays
  print(features[:2])
  print(targets[:2])
```


## Decision Tree Regressor

We are now going to use our features and targets to train a decision tree and then make a prediction. We are going to use the [DecisionTreeRegressor](http://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html) class from the `sklearn.tree` module.

The decision tree regression learning algorithm is initialised with:
```python
dtr = DecisionTreeRegressor()
```
We will discuss some optimisations later in the activity, but for now we are just going to use the default values.

To train the model, we use the fit method with the features and targets we created earlier:
```python
dtr.fit(features, targets)
```
The decision tree is now trained and ready to make a prediction:
```python
predictions = dtr.predict(features)
```
predictions is an array of predicted values corresponding to each of the input variables in the array.

Your task is to put this together for our photometric redshift data. Copy your get_features_targets from the previous problem. Use the comments to guide your implementation.

Finally, print the first 4 predictions. It should print this:

```python
[ 0.539301    0.1645703   0.04190006  0.04427702]
```
```python

import numpy as np
from sklearn.tree import DecisionTreeRegressor

# copy in your get_features_targets function here
def get_features_targets(data):
  # complete this function
  features = np.transpose([data['u']-data['g'],data['g']-data['r'],data['r']-data['i'],data['i']-data['z']])
  target = data['redshift']
  return features,target

# load the data and generate the features and targets
data = np.load('sdss_galaxy_colors.npy')
features, targets = get_features_targets(data)
  
# initialize model
dtr = DecisionTreeRegressor()
# train the model
dtr.fit(features, targets)
# make predictions using the same features
predictions = dtr.predict(features)
# print out the first 4 predicted redshifts
print(predictions[:4])
```

## Evaluating our results: accuracy

So we trained a decision tree! Great...but how do we know if the tree is actually any good at predicting redshifts?

In regression we compare the predictions generated by our model with the actual values to test how well our model is performing. The difference between the predicted values and actual values (sometimes referred to as residuals) can tell us a lot about where our model is performing well and where it is not.

While there are a few different ways to characterise these differences, in this tutorial we will use the median of the differences between our predicted and actual values. This is given by:

$$ med{\_}diff = median\left(\left| Y_{i,pred} - Y_{i,act}\right|\right) $$

Where $\left|  .....  \right|$ denotes the absolute value of the difference.



## Calculating the median difference

In this problem we will implement the function median_diff. The function should calculate the median residual error of our model, i.e. the median difference between our predicted and actual redshifts.

The median_diff function takes two arguments – the predicted and actual/target values. When we use this function later in the tutorial, these will corresponding to the predicted redshifts from our decision tree and their corresponding measured/target values. 

```python

import numpy as np
# write a function that calculates the median of the differences
# between our predicted and actual values
def median_diff(predicted, actual):
  error = np.abs(predicted - actual)
  return np.median(error)
  
```


## Evaluating our results: validation

We previously used the same data for training and testing our decision trees.

This gives an unrealistic estimate of how accurate the model will be on previously unseen galaxies because the model has been optimised to get the best results on the training data.

The simplest way to solve this problem is to split our data into training and testing subsets:

```python
# initialise and train the decision tree
dtr = DecisionTreeRegressor()
dtr.fit(train_features, train_targets)
# get a set of prediction from the test input features
predictions = dtr.predict(test_features)
# compare the accuracy of the pediction againt the actual values
print(calculate_rmsd(predictions, test_targets))
```

This method of validation is the most basic approach to validation and is called held-out validation. We will use the med_diff accuracy measure and hold-out validation in the next problem to assess the accuracy of our decision tree.


# Validating our model

In this problem, we will use median_diff from the previous question to validate the decision tree model. Your task is to complete the validate_model function.

The function should split the features and targets into train and test subsets, like this 50:50 split for features:

```python
split = features.shape[0]//2
train_features = features[:split]
test_features = features[split:]
```

Your function should then use the training split (train_features and train_targets) to train the model.

Finally, it should measure the accuracy of the model using median_diff on the test_targets and the predicted redshifts from test_features.

The function should take 3 arguments:

  - **model**: the decision tree regressor;
  - **features** - the features for the data set;
  - **targets** - The targets for the data set.

**Hint:**

keep features and targets together!
When splitting the features and targets be careful to ensure that the train_features have the correct train_targets, i.e. train_features[0] corresponds to train_targets[0] etc.

```python

import numpy as np
from sklearn.tree import DecisionTreeRegressor

# paste your get_features_targets function here
def get_features_targets(data):
  # complete this function
  features = np.transpose([data['u']-data['g'],data['g']-data['r'],data['r']-data['i'],data['i']-data['z']])
  target = data['redshift']
  return features,target

# paste your median_diff function here
def median_diff(predicted, actual):
  error = np.abs(predicted - actual)
  return np.median(error)

# write a function that splits the data into training and testing subsets
# trains the model and returns the prediction accuracy with median_diff
def validate_model(model, features, targets):
  # split the data into training and testing features and predictions
  split = features.shape[0]//2
  train_features = features[:split]
  train_targets = targets[:split]  
  test_features = features[split:]
  test_targets = targets[split:]
  # train the model
  model.fit(train_features, train_targets) 
  # get the predicted_redshifts
  predictions = model.predict(test_features)
  # use median_diff function to calculate the accuracy
  return median_diff(test_targets, predictions)

```

## Exploring the output tree

But what does the decision tree actually look like?

<img src="week5/decision_tree.jpeg",width=650, height=600 >


You can see how the decision is made at each node as well as the number of samples which reach that node. We won't go through how to make these plots in the tutorial, but you can download a demo script and data to try at home.

The median of differences of ~ 0.02. This means that half of our galaxies have a error in the prediction of < 0.02, which is pretty good. One of the reason we chose the median of differences as our accuracy measure is that it gives a fair representation of the errors especially when the distribution of errors is skewed. The graph below shows the distribution of residuals (differences) for our model along with the median and interquartile values.

<img src="week5/residuals.png",width=450, height=400 >


As you can tell the distribution is very skewed. We have zoomed in here, but the tail of the distribution goes all the way out to 6.


## The effect of training set size

The number of galaxies we use to train the model has a big impact on how accurate our predictions will be. This is the same with most machine learning methods: the more data that they are trained with, the more accurate their prediction will be.

Here is how our median difference changes with training set size:


|Training galaxies |	median_diff|
|:---:|:-----:|
|50	|0.048|
|500	|0.026|
|5000	|0.023|
|50000	|0.022|

Understanding how the accuracy of the model changes with sample size is important to understanding the limitations of our model. We are approaching the accuracy limit of the decision tree model (for our redshift problem) with a training sample size of 25,000 galaxies.

The only way we could further improve our model would be to use more features. This might include more colour indices or even the errors associated with the measured flux magnitudes.

## Before machine learning

Before machine learning, we would have tried to solve this problem with regression — by constructing an empirical model to predict how the dependent variable (redshift) varies with one or more independent variables (the colour indices).

A plot of how the colours change with redshift tells us that it is quite a complex function, for example redshift versus `u - g`:

<img src="week5/Redshift-U-G.png",width=450, height=400  >

One approach would be to construct a multi-variate non-linear regression model. Perhaps using a least squares fitting to try and determine the best fit parameters. The model would be quite complex; based on the above plot, a dampened inverse sine function would be a good starting point for such a model.

While we could try such an approach the function would be overly complex and there is no guarantee that it would yield very promising results. Another approach would be to plot a colour-index vs colour-index plot using an additional colour scale to show redshift. The following plot is an example of such a plot.

<img src="week5/Redshift-colour-colour.png",width=450, height=400  >

It shows that we get reasonably well defined regions where redshifts are similar. If we were to make a contour map of the redshifts in the colour index vs colour index space we would be able to get an estimate of the redshift for new data points based on a combination of their colour indices. This would lead to redshift estimates with significant uncertainties attached to them.

In the next problem you will re-create the Colour-index vs Colour-index plot with redshift as colour bar.

## Colour-Colour Redshift Plot

Your task here is simply to try and re-create the previous plot.

You should use the pyplot module of matplotlib which has already been imported and can be accessed through plt. In particular you can use the plt.scatter() function, with additional arguments s, c and cmap.

We are interested in the u-g and r-i colour indices.

You can make use of the plt.colorbar() function to show your scatter plots colour argument(c) to a colour bar on the side of the plot.

Make sure you implement x and y labels and give your plot a title.
Gotchas

In order to get your plot working with colours you will need to set the optional parameter lw=number. There are a lot of small points and unless we set the linewidth to zero the line surrounding each point will block out the point itself.

You should also specify the size of your plot points with the additional call arguments s=number. A reasonable place to start might be 1, but see what it looks like.

```python
import numpy as np
from matplotlib import pyplot as plt

# Complete the following to make the plot
if __name__ == "__main__":
    data = np.load('sdss_galaxy_colors.npy')
    # Get a colour map
    cmap = plt.get_cmap('YlOrRd')

    # Define our colour indexes u-g and r-i
    ug = data['u']-data['g']
    ri = data['r']-data['i']
    # Make a redshift array
    redshift = data['redshift']
    # Create the plot with plt.scatter and plt.colorbar
    plt.scatter(ug,ri, s=5, lw=0, c=redshift, cmap=cmap)
    cbar = plt.colorbar()
    cbar.ax.set_ylabel('Redshift')
    # Define your axis labels and plot title
    plt.xlabel('Colour index u - g')
    plt.title('Redshift (color) u - g versus r - i')
    plt.ylabel('Colour index r - i')
    # Set any axis limits
    plt.xlim(-0.5,2.5)
    plt.ylim(-0.5,1.0)
    plt.show()
    
```


## Summary

In this activity, we implemented some decision tree models to help predict the redshift of galaxies based on their measured colour indices. We learnt that there are several ways to assess the accuracy of the model and in our validations we used the median of the residuals.

We touched on how the problem might be approached without machine learning and found that there isn't too much available that can be simply used.

We also learnt why we need to cross validate the model and that this is done by splitting the data into seperate training and testing subsets. We will look at k-fold cross validation in the next tutorial; where the testing and training are changed to include various combinations of k seperate subsets.

In the next tutorial we will also explore how decision trees have a tendency to overfit the data.


# Week 6

## Introduction

In this activity, we will be using machine learning to classify galaxies into three types (ellipticals, spirals or galactic mergers) based on their observed properties.

In the last activity you had a go at classifying galaxies by hand on the [Galaxy Zoo website](https://www.galaxyzoo.org/#/classify) which hopefully gave you some intuition for the dataset and how we can distinguish between the different types of galaxy.

For our machine learning experiments, we are using the crowd-classified classes from Galaxy Zoo as the training data for our automatic decision tree classifier.

We'll start by looking at how classification differs from regression. We will then implement some of the new features and parameters that we will use to reduce the dimensionality of the problem. We will also show you how to measure accuracy for classification problems, before extending our classifier to use random forests.

If you would like to try this on your own machine, the data set can be downloaded [here](https://groklearning-cdn.com/modules/jSAg5N3xrpKtvpDtCtMTGV/galaxy_catalogue.npy).


## Classification vs Regression

In classification, the predictions are from a fixed set of classes, whereas in regression the prediction typically corresponds to a continuum of possible values.

In regression, we measure accuracy by looking at the size of the differences between the predicted values and the actual values. In contrast, in classification problems a prediction can either be correct or incorrect. This makes measuring the accuracy of our model a lot simpler.

In terms of implementation using decision trees, there is very little difference between classification and regression. The only notable difference is that our targets are classes rather than real values. When calculating the accuracy, we check whether the predicted class matches as the actual class.
A note on decision tree regression

In decision tree regression, the possible outputs are a finite set of values that correspond to the number of leaves/end points in the tree. Ideally we want as many points as possible to give a good approximation of the 'continuous' parameter space, whilst avoiding overfitting.

## The Galaxy Zoo data

In the last activity, you were a human classifier for the [Galaxy Zoo project](https://www.galaxyzoo.org/#/classify) and probably saw a wide range of galaxy types observed by the Sloan Digital Sky Survey. In this activity, we will limit our dataset to three types of galaxy: spirals, ellipticals and mergers.


<img src='week6/Classes.png'>

The galaxy catalogue we are using is a sample of galaxies where at least 20 human classifiers (such as yourself) have come to a consensus on the galaxy type.

Examples of spiral and elliptical galaxies were selected where there was a unanimous classification. Due to low sample numbers, we included merger examples where at least 80% of human classifiers selected the merger class.

We need this high quality data to train our classifier.

## Deciding on features

Just like in the regression activities, we need to decide on a set of key features that represent our data.

While approaches exist that determine their own feature representation and use the raw pixel values as inputs, e.g. neural networks and deep learning, the majority of existing machine learning in astronomy requires an expert to design the feature set.

In this activity we will be using a set of features derived from fitting images according to known galaxy profiles.

Most of the features we use here are based on the five observed flux magnitudes from the Sloan Digital Sky Survey filters:

<img src='week6/sdss_filters.png', width=600, height=400>


## Selecting features

The features that we will be using to do our galaxy classification are colour index, adaptive moments, eccentricities and concentrations. These features are provided as part of the SDSS catalogue.

We briefly describe these below. Further information how they are calculated can be found [here](http://skyserver.sdss.org/dr7/en/help/docs/algorithm.asp).

**Colour indices** are the same colours (u-g, g-r, r-i, and i-z) we used for regression. Studies of galaxy evolution tell us that spiral galaxies have younger star populations and therefore are 'bluer' (brighter at lower wavelengths). Elliptical galaxies have an older star population and are brighter at higher wavelengths ('redder').

**Eccentricity** approximates the shape of the galaxy by fitting an ellipse to its profile. Eccentricity is the ratio of the two axis (semi-major and semi-minor). The De Vaucouleurs model was used to attain these two axis. To simplify our experiments, we will use the median eccentricity across the 5 filters. 

<img src='week6/eccentricity_example.png', width=400, height=400>


## Selecting features - continued

Adaptive moments also describe the shape of a galaxy. They are used in image analysis to detect similar objects at different sizes and orientations. We use the fourth moment here for each band.

Concentration is similar to the luminosity profile of the galaxy, which measures what proportion of a galaxy's total light is emitted within what radius. A simplified way to represent this is to take the ratio of the radii containing 50% and 90% of the Petrosian flux.

The Petrosian method allows us to compare the radial profiles of galaxies at different distances. If you are interested, you can [read more here](http://spiff.rit.edu/classes/phys443/lectures/gal_1/petro/petro.html) on the need for Petrosian approach.

For these experiments, we will define concentration as:

$$ \textrm{conc}= \frac{\textrm{Petro}_{R50}}{\textrm{Petro}_{R90}}$$

We will use the concentration from the u, r and z bands. 


<img src='week6/concentration_example.png', width=400, height=400>


## Data description

We have extracted the SDSS and Galaxy Zoo data for 780 galaxies into a NumPy binary file. You can load the file using:
```python
import numpy as np
data = np.load('galaxy_catalogue.npy')
```

The data is a NumPy array of 780 records. Each record is a single galaxy. You can access the columns using field names. For example, to access the u-g colour filter for the first galaxy, you use:
```python
data[0]['u-g']
```

Here's a snippet that prints the field and values for the first galaxy:

```python
import numpy as np
data = np.load('galaxy_catalogue.npy')
for name, value in zip(data.dtype.names, data[0]):
  print('{:10} {:.6}'.format(name, value))
```

It shows the four colour features, median eccentricity, fourth adaptive moment of each filter, the Petrosian fluxes at a radius of 50% and 90% for the u, r and z filters, and finally the class:
``` python
    u-g        1.85765
    g-r        0.67158
    r-i        0.4231
    i-z        0.3061
    ecc        0.585428
    m4_u       2.25195
    m4_g       2.33985
    m4_r       2.38065
    m4_i       2.35974
    m4_z       2.39553
    petroR50_u 3.09512
    petroR50_r 3.81892
    petroR50_z 3.82623
    petroR90_u 5.17481
    petroR90_r 8.26301
    petroR90_z 11.4773
    class      merger
```

NumPy also allows you to access a field for all of the rows at once, i.e. a column, using the field's name:
```python
data['u-g']
```


## Splitting the train and test sets

To start, we need to split the data into training and testing sets.

Your task is to implement the `splitdata_train_test` function. It takes a NumPy array and splits it into a training and testing NumPy array based on the specified training fraction. The function takes two arguments and should return two values:

### Arguments

- `data`: the NumPy array containing the galaxies in the form described in the previous slide;
- `fraction_training`: the fraction of the data to use for training. This will be a float between 0 and 1.

The number of training rows should be truncated if necessary. For example, with a fraction of 0.67 and our 780 galaxies, the number of training rows is 780*0.67 = 722.6, which should be truncated to 722 using int. The remaining rows should be used for testing.

### Return values

- `training_set`: the first value is a NumPy array training set;
- `testing_set`: the second value is a NumPy array testing set.

Using the supplied driver code, and our input data and a fraction of 0.7, the program should print the following values:

```python
Number data galaxies: 780
Train fraction: 0.7
Number of galaxies in training set: 546
Number of galaxies in testing set: 234
```

### Good practice: randomize the dataset order

You shouldn't assume that the data has already been shuffled. If you look at `data['class']` you will see that the `merger`, `elliptical` and `spiral` examples appear together. Failing to shuffle the data will produce a very bad classifier! You can use:

```python
np.random.seed(0)
np.random.shuffle(data)
```
The first statement ensures the shuffle is the same for each experiment, so you get consistent results, the second shuffles the rows of the data array in place.

```python
import numpy as np
def splitdata_train_test(data, fraction_training):
  np.random.seed(0)
  np.random.shuffle(data)
  rows = int(len(data)*fraction_training)
  return data[:rows], data[rows:]

if __name__ == "__main__":
  data = np.load('galaxy_catalogue.npy')
  # set the fraction of data which should be in the training set
  fraction_training = 0.7
  # split the data using your function
  training, testing = splitdata_train_test(data, fraction_training)
  # print the key values
  print('Number data galaxies:', len(data))
  print('Train fraction:', fraction_training)
  print('Number of galaxies in training set:', len(training))
  print('Number of galaxies in testing set:', len(testing))
```

## Generating features and targets

Next, we generate features and targets for the decision tree.

The `generate_features_targets` function is mostly complete. However, you need to calculate the concentration values for the u, r and z filters.

Your task is to calculate the concentration for each filter from the 50% and 90% Petrosian radius measurements:

$$ \textrm{conc}= \frac{\textrm{Petro}_{R50}}{\textrm{Petro}_{R90}}$$


As described earlier, data has the following fields:

- **colours**: u-g, g-r, r-i, and i-z;
- **eccentricity**: ecc
- **4th adaptive moments**: m4_u, m4_g, m4_r, m4_i, and m4_z;
- **50% Petrosian**: petroR50_u, petroR50_r, petroR50_z;
- **90% Petrosian**: petroR90_u, petroR90_r, petroR90_z.


```python
def generate_features_targets(data):
  # complete the function by calculating the concentrations
  targets = data['class']
  features = np.empty(shape=(len(data), 13))
  features[:, 0] = data['u-g']
  features[:, 1] = data['g-r']
  features[:, 2] = data['r-i']
  features[:, 3] = data['i-z']
  features[:, 4] = data['ecc']
  features[:, 5] = data['m4_u']
  features[:, 6] = data['m4_g']
  features[:, 7] = data['m4_r']
  features[:, 8] = data['m4_i']
  features[:, 9] = data['m4_z']
  # fill the remaining 3 columns with concentrations in the u, r and z filters
  # concentration in u filter
  features[:, 10] = data['petroR50_u']/data['petroR90_u']
  # concentration in r filter
  features[:, 11] = data['petroR50_r']/data['petroR90_r'] 
  # concentration in z filter
  features[:, 12] =  data['petroR50_z']/data['petroR90_z']
  return features, targets
```

## Train the decision tree classifier

It is time to use the functions we wrote to split the data and generate the features, and then train a decision tree classifier.

Your task is complete the `dtc_predict_actual` function by following the Python comments. The purpose of the function is to perform a held out validation and return the predicted and actual classes for later comparison.

The function takes a single argument which is the full data set and should return two NumPy arrays containing the predicted and actual classes respectively.

You will also need to copy your solutions from the previous two questions into the spaces allocated.

```python
def dtc_predict_actual(data):
  # split the data into training and testing sets using a training fraction of 0.7
  fraction_training = 0.7
  training, testing = splitdata_train_test(data, fraction_training)
  # generate the feature and targets for the training and test sets
  # i.e. train_features, train_targets, test_features, test_targets
  train_features,train_targets = generate_features_targets(training)
  test_features, test_targets = generate_features_targets(testing) 
  # instantiate a decision tree classifier
  model = DecisionTreeClassifier()
  # train the classifier with the train_features and train_targets
  model.fit(train_features,train_targets)
  # get predictions for the test_features
  predictions = model.predict(test_features)
  # return the predictions and the test_targets
  return predictions, test_targets

if __name__ == '__main__':
  data = np.load('galaxy_catalogue.npy')
  predicted_class, actual_class = dtc_predict_actual(data)
  # Print some of the initial results
  print("Some initial results...\n   predicted,  actual")
  for i in range(10):
    print("{}. {}, {}".format(i, predicted_class[i], actual_class[i]))
```


## Accuracy and model score

The accuracy of classification problems is a lot simpler to calculate than for regression problems. The simplest measure is the fraction of objects that are correctly classified. That is


$$ \textrm{accuracy}= \frac{\textrm{# correct predictions}}{\textrm{# predictions}}$$

$$ \textrm{accuracy}= \frac{\sum_{i=1}^{n}\textrm{predicted}_{i} = \textrm{actual}_{i}}{n}$$


The accuracy measure is often called the model score. While the way of calculating the score can vary depending on the model, the accuracy is the most common for classification problems.

Note: `sklearn` has methods to get the model score. Most models will have a `score` method which in the case of the decision tree classifier uses the above formula. The `cross_val_score` function in the `model_selection` module can be used to get k cross validated scores.


## Confusion matrices

In addition to an overall accuracy score, we'd also like to know where our model is going wrong. For example, were the incorrectly classified mergers mis-classified as spirals or ellipticals? To answer this type of question we use a confusion matrix. An example confusion matrix for our problem is shown below:


<img src='week6/example_confusion.png', width=400, height=400>

The x axis represents the predicted classes and the y axis represents the correct classes. The value in each cell is the number of examples with those predicted and actual classes. Correctly classified objects are along the diagonal of the matrix.

So of the 260 actual spirals (correct class) in the data set, 198 are correctly predicted as spirals, 5 are incorrectly predicted as ellipticals and 57 are incorrectly predicted as mergers.

The sum along each row or column can be used to get the totals of true and predicted classes. So for example, by summing each of the rows we can confirm that there are 260 mergers, 260 spirals and 260 ellipticals in the data set.


## Accuracy in classification

Your task is to complete the calculate_accuracy function. The function should calculate the accuracy: the fraction of predictions that are correct (i.e. the model score):

$$ \textrm{accuracy}= \frac{\textrm{# correct predictions}}{\textrm{# predictions}}$$

The function takes two arguments;

- predicted: an array of the predicted class for each galaxy.
- actual: an array of the actual class for each galaxy.

The return value should be a float (between 0 and 1).

```python
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from support_functions import plot_confusion_matrix, generate_features_targets

# Implement the following function
def calculate_accuracy(predicted, actual):
  correct = predicted[predicted == actual]
  return float(len(correct)/len(actual))

if __name__ == "__main__":
  data = np.load('galaxy_catalogue.npy')

  # split the data
  features, targets = generate_features_targets(data)

  # train the model to get predicted and actual classes
  dtc = DecisionTreeClassifier()
  predicted = cross_val_predict(dtc, features, targets, cv=10)

  # calculate the model score using your function
  model_score = calculate_accuracy(predicted, targets)
  print("Our accuracy score:", model_score)

  # calculate the models confusion matrix using sklearns confusion_matrix function
  class_labels = list(set(targets))
  model_cm = confusion_matrix(y_true=targets, y_pred=predicted, labels=class_labels)

  # Plot the confusion matrix using the provided functions.
  plt.figure()
  plot_confusion_matrix(model_cm, classes=class_labels, normalize=False)
  plt.show()
```

## Random forests classification

So far we have used a single decision tree model. However, we can improve the accuracy of our classification by using a collection (or ensemble) of trees as known as a *random forest*.

A random forest is a collection of decision trees that have each been independently trained using different subsets of the training data and/or different combinations of features in those subsets.

When making a prediction, every tree in the forest gives its own prediction and the most common classification is taken as the overall forest prediction (in regression the mean prediction is used).


## Advantages of random forest classifiers

Random forests help to mitigate overfitting in decision trees.

**Training data** is spread across decision trees. The subsets are created by taking random samples with replacement. This means that a given data point can be used in several subsets. (This is different from the subsets used in cross validation where each data point belongs to one subset).

**Individual trees** are trained with different subsets of features. So in our current problem, one tree might be trained using eccentricity and another using concentration and the 4th adaptive moment. By using different combinations of input features you create expert trees that are can better identify classes by a given feature.

**The sklearn random forest only uses the first form of sampling.**


## Random Forest

Your task here is to complete the `rf_predict_actual` function. It returns the predicted and actual classes for our galaxies using a random forest 10-fold with cross validation.

You should use the `RandomForestClassifier` class from the `sklearn.ensemble` module. It can be instantiated with:
```python
rfc = RandomForestClassifier(n_estimators=n_estimators)
```

`n_estimators` is the the number of decision trees in the forest.

`rf_predict_actual` takes two arguments: the `data` used throughout this activity and the number of estimators (`n_estimators`) to be used in the random forest.

The function should return two NumPy arrays containing the predicted and actual classes respectively.

You can copy and paste the functions from previous questions. However, we have provided the `generate_features_targets` function in the support library.

Use the `cross_val_predict` function from the model_selection module as we did in the last question.

You can read its documentation [here](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html). This approach allows us to get a prediction for every galaxy in the data set through cross validation. It also means that we don't need to manage the training and test sets.

```python
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from support_functions import generate_features_targets, plot_confusion_matrix, calculate_accuracy

# complete this function to get predictions from a random forest classifier
def rf_predict_actual(data, n_estimators):
  # generate the features and targets
  features, targets = generate_features_targets(data)
  # instantiate a random forest classifier using n estimators
  rfc = RandomForestClassifier(n_estimators=n_estimators)
  # get predictions using 10-fold cross validation with cross_val_predict
  y_pred = cross_val_predict(rfc, features, targets)
  # return the predictions and their actual classes
  return y_pred, targets

if __name__ == "__main__":
  data = np.load('galaxy_catalogue.npy')

  # get the predicted and actual classes
  number_estimators = 50              # Number of trees
  predicted, actual = rf_predict_actual(data, number_estimators)

  # calculate the model score using your function
  accuracy = calculate_accuracy(predicted, actual)
  print("Accuracy score:", accuracy)

  # calculate the models confusion matrix using sklearns confusion_matrix function
  class_labels = list(set(actual))
  model_cm = confusion_matrix(y_true=actual, y_pred=predicted, labels=class_labels)

  # plot the confusion matrix using the provided functions.
  plt.figure()
  plot_confusion_matrix(model_cm, classes=class_labels, normalize=False)
  plt.show()

```

## Results discussion

Did the random forest improve the accuracy of the model? The answer is yes – we see a substantial increase in accuracy. When we look at the 10-fold cross validation results, we see that the random forest systematically out performs a single decision tree:

|   |Random Forest	|Decision Tree|
|-----  |:--:|:---:| 
|Median score	|0.865	|0.775|
|Mean score	|0.867	|0.792|
|Standard deviation of scores	|0.036	|0.035|

The random forest is around ~6-7% more accurate than a standard decision tree.

Below is a side by side comparision of the confusion matrices from our galaxy catalogues. The first showing the results for the random forest classifier and the second for the decision tree classifier. There are improvements accross the board with the biggest improvement (percentage) being between ellipticals and spirals.

<img src='week6/forest_tree_comparison.png', width=600, height=600>






# Conclusions
In this activity we have used decision tree classifiers to identify a galaxies type by a selection of derived features (e.g. eccentricity and concentration).

We have learnt that asessing the models performance is a lot simpler with classification than it is with regression.

Confusion matrices can be a useful tool to help understand where our model is over and under preforming with respect to each class.

We started out with the goal of using decision trees to classify galaxies as one of three types. We found that we were able to achieve an accuracy of around 80% using decision trees which is very good when you consider the statistical accuracy of a random selection is 33% and the naive approach using only pixel values yielded ~64%.

We were able to improve on the accuracy of the decision tree classifier by using a selection of them in ensemble learning with a random forest.


# Congratulations!

Congratulations, you've finished the final set of activities for the MOOC!

If you've still got questions about any of the content, head to the forums to discuss with your fellow learners.

Now head back to Coursera for the wrap up lecture. Please let us know if you have any feedback about the Grok Learning platform.